In [ ]:
import matplotlib

# 从numpy库导入所有函数和类（不推荐在实际项目中使用这种方式）
from numpy import *

# 从sklearn（scikit-learn）库导入所有函数和类（不推荐在实际项目中使用这种方式）
from sklearn import *

# 从scipy库导入stats（统计函数）和special（特殊数学函数）模块，是Numpy的高级版
from scipy import stats, special


In [ ]:
# 提取文本数据集

removeset = ('headers', 'footers', 'quotes') # 移除标题、页脚和引用内容

# 定义我们只使用的4个新闻类别
cats      = ['alt.atheism',         # 关于无神论
             'talk.religion.misc',  # 关于宗教
             'comp.graphics',       # 关于计算机图形学
             'sci.space']           # 关于太空科学

# 加载训练数据集
# subset='train' 表示加载训练集
# remove=removeset 表示移除指定的内容（标题、页脚、引用）
# categories=cats 表示只加载指定的4个类别
# data_home='./' 表示数据存储在当前目录下
newsgroups_train = datasets.fetch_20newsgroups(subset='train',
                           remove=removeset, categories=cats, data_home='./')


newsgroups_test  = datasets.fetch_20newsgroups(subset='test', 
                           remove=removeset, categories=cats, data_home='./')

In [ ]:
# 处理数据
traindata = newsgroups_train.data
trainY = newsgroups_train.target
testdata = newsgroups_test.data
testY  = newsgroups_test.target

In [ ]:
# 文本是 “非结构化数据”，需要先转换为模型可理解的 “结构化数值特征”（词汇表是实现这一转换的核心工具之一）。
from sklearn import feature_extraction
# BoW词汇表
cntvect = feature_extraction.text.CountVectorizer(stop_words='english', max_features=100)
trainX = cntvect.fit_transform(traindata)
testX  = cntvect.transform(testdata)

# 查看特征形状
print(f"训练集特征形状: {trainX.shape}")
print(f"测试集特征形状: {testX.shape}")

In [ ]:
# 朴素伯努利贝叶斯
# 用伯努利是因为文本特征是0/1二值型的

from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score

bmodel=BernoulliNB()
#bnb.fit接受（二维矩阵，一维数组）
bmodel.fit(trainX,trainY)
testY_pred=bmodel.predict(testX)
accuracy=accuracy_score(testY,testY_pred)
accuracy

In [ ]:
# 多项式朴素贝叶斯
# 用多项式是因为文本特征是计数型的
from sklearn.naive_bayes import MultinomialNB
mmodel = MultinomialNB()
mmodel.fit(trainX, trainY)
predY= mmodel.predict(testX)
accuracy=accuracy_score(testY,predY)
accuracy

In [ ]:
#生成TF-IDF特征
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfvect = TfidfVectorizer(
    stop_words='english', 
    max_features=100
)
trainX_tf= tfidfvect.fit_transform(traindata)  # 训练集TF-IDF特征
testX_tf = tfidfvect.transform(testdata)        # 测试集TF-IDF特征

In [ ]:
#训练模型
mmodel_tf = MultinomialNB()
mmodel_tf.fit(trainX_tf, trainY)
predY_tf = mmodel_tf.predict(testX_tf)
accuracy=accuracy_score(testY,predY_tf)
accuracy

In [ ]:
# 泊松朴素贝叶斯
from scipy.stats import poisson
from scipy.special import logsumexp
import numpy as np

class PoissonBayes:
    def __init__(self):
        pass
    
    def fit(self, X, y):
        # 获取唯一类别数量
        self.K = len(np.unique(y))
        self.mu = []       # 存储每个类别每个特征的均值
        self.pi = []       # 存储先验概率
        
        for c in range(self.K):
            # 筛选出属于当前类别c的所有样本
            # 注意：X是稀疏矩阵，需要转换为数组才能正确计算均值
            Xc = X[y == c].toarray() if hasattr(X, 'toarray') else X[y == c]
            # 计算每个特征的均值（按列计算）
            self.mu.append(np.mean(Xc, axis=0))
            # 计算先验概率
            self.pi.append(np.mean(y == c))
        
        self.pi = np.array(self.pi)

    # 计算类别c的类条件概率log p(x|y=c)的对数
    def compute_logccd(self, X, c):
        # 将稀疏矩阵转换为数组以便处理
        if hasattr(X, 'toarray'):
            X = X.toarray()
            
        lx = []
        # 获取当前类别的均值数组
        mu_c = self.mu[c]
        for i in range(X.shape[1]):
            # 获取单个特征的均值（标量）
            mu_ic = mu_c[i]
            # 获取单个特征的所有样本值
            x_i = X[:, i]
            
            # 处理均值为零的情况
            if mu_ic == 0:
                # 如果均值为零，只有当x_i为零时概率才为1
                log_p = np.where(x_i == 0, 0, -np.inf)
            else:
                # 计算泊松分布的对数概率密度
                log_p = poisson.logpmf(x_i, mu_ic)
                
            lx.append(log_p)
        
        # 对所有特征的对数概率求和
        return np.sum(lx, axis=0)
    
    # 计算联合似然/概率的对数: log p(x,y=c) = log p(x|y=c) + log p(y=c)
    # 即贝叶斯定理公式右侧的分子（联合概率）
    def compute_logjoint(self, X):
        jl = [] # 列表：每个元素是一个长度为 “样本数” 的数组（对应一个类别的联合对数概率）
        for c in range(self.K):
            jl.append(self.compute_logccd(X, c) + np.log(self.pi[c]))
        p = np.stack(jl, axis=-1)#axis=-1即多维数组里最后那一列
        #p[i, c] 表示第 i 个样本属于类别 c 的联合对数概率，即log(p(xi|y=c))
        return p
    
    # 计算每个类别给定X的后验对数概率 log p(y|x)
    def predict_logproba(self, X):
        # 计算联合对数似然 log p(x,y)
        lp = self.compute_logjoint(X)
        
        # 计算log p(x) = log （所有类别的联合概率之和）
        lpx = logsumexp(lp, axis=1)
        
        # 后验对数概率公式: log p(y|x) = log p(x,y) - log p(x)
        # lpx[:, np.newaxis]用于维度扩展，使广播运算成立
        # 广播运算：让形状不一样但 “兼容” 的数组，能自动调整成相同维度后再做计算
        return lp - lpx[:, np.newaxis]
    
    # 计算每个类别给定x的后验概率 p(y|x)
    def predict_proba(self, X):
        # 对后验对数概率取指数，转换为概率值
        return np.exp(self.predict_logproba(X))
    
    # 预测x的最可能类别
    def predict(self, X):
        # 联合对数概率矩阵：[样本数, 类别数]，例如：[-2.1, -3.2, -1.8],  样本对类别0、1、2的联合对数概率
        lp = self.compute_logjoint(X)
        # argmax(lp, axis=1)沿行方向（每个样本）找最大值的索引
        c = np.argmax(lp, axis=1)
        
        # 返回预测的类别标签
        return c
    

In [ ]:
pnb=PoissonBayes()
pnb.fit(trainX,trainY)
predY = pnb.predict(testX)
acc = metrics.accuracy_score(testY, predY)
acc